# Generating Synthetic Text

In this exercise we try to use character ngrams to generate synthetic text, see "künstliche Sprache" (Kupfmüller).

The main idea is to establish a ngram statistic of a given corpus. The frequencies of the ngrams is assumed to approximates the probability of a specific ngram to occurr in any given text of that language. We can use this to generate sequences of characters from any given seed.

We can use this assumption to generate a sequence of character by assuming that certain ngrams are more likely than others.


Moreover, if we have the ngram statistic and see an "(n-1)-gram" $t$, we can restrict the established ngram statistic to only matching ngrams that start with $t$. The probabilites in the resulting list can be renormalized and sampled from


#### The algorithm is:
   1. Establish the $n$-gram probabilities for the top $k$ ngrams overall of a corpus (ngram with corresponding probability).
   2. For a given string $s$ predict the next character (see next algorithm)
   3. Repeat step two to generate as many next characters as you want
   
#### Generate a character by:
   1. Take the last $(n-1)$ characters of $s$ as substring $t$
   2. Restrict the known ngrams to ngrams that start with $t$
   3. Sample from this list of ngrams according to the probabilities
   4. Take the last character of this ngram and add to initial string
   
#### Ressources
As a basis you can again use the books corpus as a whole.
This time, for the creation of synthetic text it is important that the corpus contains spaces and punctuation. Otherwise the resulting text will contain no spaces!

Also for the sampling process, have a look at `numpy.random.choice` which allows you to put in a list of symbols and a list of corresponding probabilities and returns an element according to them. 


#### Deliberations:
The crucial part of this is the sampling process in step 3.
What would be the problem with just returning the ngram with the highest probability?

What is the influence of the size of $n$?

What is the influence of $k$?

What are some restrictions of this approach?

In [ ]:
import nltk
import numpy as np
import string

In [ ]:
english_text = nltk.corpus.gutenberg.raw(nltk.corpus.gutenberg.fileids())#[:200000]
print(english_text[100:150])

In [ ]:
c = nltk.FreqDist(nltk.ngrams(english_text.lower(), 3)).most_common(40) 
print(c)
grams = ["".join(x[0]) for x in c]
counts = np.array([x[1] for x in c])

In [ ]:
print(grams)
print(counts)
print(counts/counts.sum())

In [ ]:
"north"[-2:]

In [ ]:
indices = [i for i, gram in enumerate(grams) if gram.startswith("th")] 
indices 

In [ ]:
probs = counts[indices]
print(probs)
probs = probs / probs.sum()
probs

In [ ]:
selection = np.random.choice(indices, p=probs)
selection

In [ ]:
grams[selection][-1]

### Everything in one class:

In [ ]:
class LanguageGenerator:
    def __init__(self,texts, n=5, topk=10000):
        c = nltk.FreqDist(nltk.ngrams(texts.lower(), n)).most_common(topk) 
        
        self.n = n
        self.grams = ["".join(x[0]) for x in c]
        self.counts = np.array([x[1] for x in c]) # saving the probabilities as numpy array.
        self.counts = self.counts / self.counts.sum()
    
    
    def generate(self,text):
        """Generate the next character in the sequence"""
        
        # Get last n-1 characters of the input
        prefix = text[-(self.n-1):]
        
        # Find all the ngrams starting with the prefix
        indices = [i for i, gram in enumerate(self.grams) if gram.startswith(prefix)] 
        
        if len(indices) == 0:
            # If there are no ngrams starting with the ngrams
            character = np.random.choice(list(string.ascii_lowercase))
            text = text + character
        else:
            probs = self.counts[indices]
            probs = probs / probs.sum() # Renormalize

            # Sample
            selection = np.random.choice(indices, p=probs)
            #selection = np.argmax(indices)
            #print(selection)
        
            text = text + self.grams[selection][-1]
        return text
    
    
    
    def generate_n(self, text, n = 100):
        for _ in range(n):
            text = self.generate(text)
        return text

In [ ]:
langen = LanguageGenerator(english_text, n=5, topk=100000)  

In [ ]:
langen.generate("Hello my namy is Alice")

In [ ]:
text = "Hello my namy is Alice"
#text = "Yesterday there was a"

text = langen.generate_n(text, 1000)
print(text)